In [4]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

import os
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

/bin/bash: /home/themandalorian/anaconda3/envs/dl-gpu/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [284]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [285]:
# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']

In [286]:
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])
#train_labels = to_categorical(train_labels)
#test_labels = to_categorical(test_labels)

# Load the CIFAR-100 dataset
data_path = 'cifar-100-python/'
train_data_100 = unpickle(data_path + 'train')[b'data']
train_data_100 = train_data_100.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data_100 = unpickle(data_path + 'test')[b'data']
test_data_100 = test_data_100.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels_100 = np.array(unpickle(data_path + 'train')[b'fine_labels'])
test_labels_100 = np.array(unpickle(data_path + 'test')[b'fine_labels'])

In [287]:
# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000


In [288]:
# Print the number of features, classes, and training samples for CIFAR-100
print("\nCIFAR-100")
print("Number of features:", train_data_100.shape[1:])
print("Number of classes:", len(np.unique(train_labels_100)))
print("Number of training samples:", train_data_100.shape[0])


CIFAR-100
Number of features: (32, 32, 3)
Number of classes: 100
Number of training samples: 50000


In [289]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
train_data_100, train_labels_100 = shuffle(train_data_100, train_labels_100, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)
test_data_100, test_labels_100 = shuffle(test_data_100, test_labels_100, random_state=42)

# Get the number of classes in the dataset
#num_classes = len(np.unique(train_labels))

# Initialize empty lists to store the indices of the selected examples
#selected_train_indices = []
#selected_test_indices = []

# Loop over each class
#for i in range(num_classes):
    # Get the indices of the examples belonging to the current class in the training dataset
#    class_train_indices = np.where(train_labels == i)[0]
    # Randomly choose 10% examples from the current class in the training dataset
#    selected_train_indices.extend(np.random.choice(class_train_indices, size=int(len(class_train_indices)*0.1), replace=False))

    # Get the indices of the examples belonging to the current class in the test dataset
#    class_test_indices = np.where(test_labels == i)[0]
    # Randomly choose 10% examples from the current class in the test dataset
#    selected_test_indices.extend(np.random.choice(class_test_indices, size=int(len(class_test_indices)*0.1), replace=False))

# Select the 10% examples from the training and test datasets using the selected indices
#train_data = train_data[selected_train_indices]
#train_labels = train_labels[selected_train_indices]
#test_data = test_data[selected_test_indices]
#test_labels = test_labels[selected_test_indices]

# Print the number of features, classes, and training samples for CIFAR-10
#print("CIFAR-10")
#print("Number of features:", train_data.shape[1:])
#print("Number of classes:", len(np.unique(train_labels)))
#print("Number of training samples:", train_data.shape[0])


In [290]:
# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0
x_train_100 = train_data_100.astype('float32') / 255.0
x_test_100 = test_data_100.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)
y_train_100 = to_categorical(train_labels_100, num_classes=100)
y_test_100 = to_categorical(test_labels_100, num_classes=100)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)
# fit model
steps = int(x_train.shape[0] / 64)

In [291]:
#Build ResNet-50 model
base_model = ResNet50(include_top=False, weights=None, input_shape=(32, 32, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model_10 = Model(inputs=base_model.input, outputs=predictions)

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10.{epoch:02d}.h5', save_freq='epoch')

# Define a learning rate schedule function
def lr_schedule(epoch):
    """
    Returns a learning rate based on the epoch number.
    """
    learning_rate = 0.001
    if epoch > 2:
        learning_rate = 0.00075
    if epoch > 4:
        learning_rate = 0.0005
    if epoch > 6:
        learning_rate = 0.00025
    if epoch > 8:
        learning_rate = 0.000125
    if epoch > 8:
        learning_rate = 0.0000625
    if epoch > 10:
        learning_rate = 0.00003125
    if epoch > 12:
        learning_rate = 0.000015
    if epoch > 15:
        learning_rate = 0.0000075
    return learning_rate

# Create a LearningRateScheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

model_10.compile(optimizer=Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])

In [293]:
from tensorflow.keras.models import load_model

learning_rate = 0.000125

# Define a learning rate schedule function
def lr_schedule_new(epoch):
    """
    Returns a learning rate based on the epoch number.
    """
    learning_rate = 0.000125
    if epoch >= 1:
        learning_rate = 0.000125
    if epoch >= 3:
        learning_rate = 0.0000625
    if epoch >= 5:
        learning_rate = 0.00003125
    if epoch >= 7:
        learning_rate = 0.000015
    if epoch >= 9:
        learning_rate = 0.0000075
    return learning_rate

# Create a LearningRateScheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule_new)

# load the pre-saved model
model_10 = load_model('model_10.09.h5')

model_10.compile(optimizer=Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model.{epoch:02d}.h5', save_freq='epoch')

# Model Fit
model_10.fit(it_train, steps_per_epoch=steps, epochs=11, validation_data=(x_test, y_test), 
             callbacks=[lr_scheduler,checkpoint_callback])

Epoch 1/11
781/781 [==============================] - 724s 916ms/step - loss: 0.7830 - accuracy: 0.7259 - val_loss: 0.8284 - val_accuracy: 0.7152 - lr: 1.2500e-04
Epoch 2/11
781/781 [==============================] - 697s 892ms/step - loss: 0.7381 - accuracy: 0.7429 - val_loss: 1.0076 - val_accuracy: 0.6696 - lr: 1.2500e-04
Epoch 3/11
781/781 [==============================] - 698s 894ms/step - loss: 0.7049 - accuracy: 0.7522 - val_loss: 0.7491 - val_accuracy: 0.7462 - lr: 1.2500e-04
Epoch 4/11
781/781 [==============================] - 712s 912ms/step - loss: 0.6501 - accuracy: 0.7715 - val_loss: 0.6858 - val_accuracy: 0.7677 - lr: 6.2500e-05
Epoch 5/11
781/781 [==============================] - 709s 908ms/step - loss: 0.6310 - accuracy: 0.7798 - val_loss: 1.1641 - val_accuracy: 0.6407 - lr: 6.2500e-05
Epoch 6/11
781/781 [==============================] - 723s 925ms/step - loss: 0.6158 - accuracy: 0.7859 - val_loss: 0.6607 - val_accuracy: 0.7726 - lr: 3.1250e-05
Epoch 7/11
781/781 [==

In [295]:
# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

313/313 [==============================] - 25s 78ms/step - loss: 0.6484 - accuracy: 0.7809
Test accuracy: 0.7809000015258789
